## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

## Import the data.  

While viewing the data with Data Wrangler I notice that the data column is not the appropriate type. Therefore, while importing I will change the InvoiceDate to type datetime64.

In [ ]:
def clean_data(df):
    # Change column type to datetime64[ns] for column: 'InvoiceDate'
    df = df.astype({'InvoiceDate': 'datetime64[ns]'})
    # Split the InvoiceDate column into year, month, day, and hour columns
    return df

# Loaded Excel file into dataframe
df = pd.read_excel(r'd:\OneDrive\Business\Data Science\WS-Default\Data\Online Retail.xlsx')

df = clean_data(df.copy())
df.head()

List the columns and store for future use

In [ ]:
df_col = df.columns
df_col

In [ ]:
df.describe()

From this describe it look as if there are order cancellations due to the negative values in the Quantity column. Also, there may be some errors in the pricing of some units as there are more negative values in UnitPrice.

Double check of Dtypes and also looking for null values

In [ ]:
df.info()

There are definitely some null, let's make it easier to see.  

In [ ]:
df.isnull().any()

I'm going to look to see if these missing values are distributed evenly.

In [ ]:
import missingno as msno
msno.matrix(df)

It appears that the missing CustomerID values are randomly distributed throughout the data. I'm going to separate it from the main data into a separate dataframe.

In [ ]:
CustomerID_null = df[df['CustomerID'].isnull()]
CustomerID_null

In [ ]:
# export to csv for safe keeping
CustomerID_null.to_csv('CustomerID_null.csv', index=False)

In [ ]:
# Drop rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])
df

In [ ]:
# recheck missing values
df.isnull().any()

I noticed that CustomerID is stored as a float64. However, looks like CustomerID should be int32 as it's more of a category that a number.  

I will check to see if there are any numbers after the decimal. If not, then I will change the dtype.

In [ ]:
fractions, _ = np.modf(df['CustomerID'])
has_decimal = fractions != 0
has_decimal.value_counts()

In [ ]:
df['CustomerID'] = df['CustomerID'].astype('int')
df

Let's see where we now stand.

In [ ]:
df.info()

In [ ]:
df.describe()

## Negative Values

### UnitPrice negative values disapeared with the removal of CustomerID NaN values.

### Quantity negative values

In [ ]:
neg_quantity = df[df['Quantity'] < 0]
neg_quantity

I'm going to export this to a cvs file for safe keeping.

In [ ]:
neg_quantity.to_csv('neg_quantity.csv', index=False)

In [ ]:
df = df[df['Quantity'] >= 0]
df

I noticed some product descriptions when I looked at the neg_quantity dataframe. Let's look at all the descriptions with 2 words and fewer.

In [ ]:
des_small = df[df['Description'].str.count(' ') < 2]
look_des_small = des_small['Description'].value_counts()
look_des_small

There seem to be a quite a few entries that are not products. I will remove and store them in a separate dataframe.

In [ ]:
nonproducts = df[df['Description'].isin(['POSTAGE', 'Manual', 'Bank Charges', 'DOTCOM POSTAGE'])]
nonproducts

In [ ]:
nonproducts.to_csv('nonproducts.csv', index=False)

In [ ]:
df = df[~df['Description'].isin(['POSTAGE', 'Manual', 'Bank Charges', 'DOTCOM POSTAGE'])]
df

Let's see where we stand now.

In [ ]:
df.describe()

It looks like there are some extreme outliers. I will take a look at them.

In [ ]:
df_q_outlier = df[df['Quantity'] > 100]
df_q_outlier

I will now compare this dataframe to the neg_quantity dataframe for possible canceled orders.

In [ ]:
neg_quantity['Quantity'] = neg_quantity['Quantity'].abs()

df_q_outlier['MATCH'] = (df_q_outlier['StockCode'].isin(neg_quantity['StockCode']) &
                        df_q_outlier['Quantity'].isin(neg_quantity['Quantity']))

df_q_outlier


In [ ]:
cancelled_orders = df_q_outlier[df_q_outlier['MATCH']]

cancelled_orders = cancelled_orders.drop('MATCH', axis=1)

cancelled_orders

In [ ]:
df = df.drop(cancelled_orders.index)

In [ ]:
df.describe()

It seems we've knocked out a good portion of outliers as cancelled orders. But there is still a large quantity order remaining. I will see if that order was in the df_q_outlier dataframe and if it was a cancelled order.

In [ ]:
print(df_q_outlier[df_q_outlier['Quantity'] == 12540])

Interesting, a price of 0.0. Maybe it was a free giveaway. Or a pricing error that was exploited.

It appears that UnitPrice seems reasonable now that we've filtered out non-products and cancelled orders. But let's take a look.

In [ ]:
df_u_outlier = df[df['UnitPrice'] > 100]
df_u_outlier

# This is the end of the cleaning portion. 
### I'll export the result to be used in further analysis and models.

In [ ]:
df.to_csv('data_cleaned.csv', index=False)